<a href="https://colab.research.google.com/github/gravity0717/Capstone/blob/main/MuitiCNN_MU_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import pandas as pd 
import json 
import tarfile 
import torch 
from torch.utils.data import Dataset
import math
from PIL import Image
from google.colab import drive 
drive.mount('/content/drive')
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
from sklearn.metrics import mean_squared_error
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [76]:
from PIL import Image 
import glob
from torch.utils.data.dataset import Dataset
import numpy as np 
from torchvision import transforms
import os 
class MU(Dataset):
  def __init__(self,csv_path):
    self.to_tensor=transforms.ToTensor()
    # Read the csv file 
    self.data_info = pd.read_csv(csv_path,header=None)
    # Read image path 
    self.img_path =  np.asarray(self.data_info.iloc[1:,6])
    # Read the log_view < log_view as label>
    self.label_arr =np.asarray(self.data_info.iloc[1:,7])
    
    
    #data augmentation 을 기대할 수 있는 부분 ???! 고민해보자 
    self.transformations = \
            transforms.Compose([transforms.Resize(256),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                transforms.ToTensor()])
    
  
        
  def __len__(self):
    return len(self.label_arr)
  # def wait to drop 

  def __getitem__(self,idx):
   #Get image name (actually path where images are )from pandas 
    if self.img_path is not np.nan:
      u=self.img_path[idx] + "/u" # 상의
      p=self.img_path[idx] + "/p" # 하의
      s=self.img_path[idx] + "/s" # 상의
    else: 
      pass 
    # Open Image 
    # 이미지가 경로에 없는 겨우 대비해야 할 것 
    if os.path.isfile(u):
      u_as_img =Image.open(u)
    else :
      u_as_img =Image.new("RGB", (256, 256), (255, 255, 255))
    if os.path.isfile(p):
      p_as_img =Image.open(p)
    else :
      p_as_img =Image.new("RGB", (256, 256), (255, 255, 255))

    if os.path.isfile(s):
      s_as_img =Image.open(s)
    else :
      s_as_img =Image.new("RGB", (256, 256), (255, 255, 255))
    #Img -> to tensor 
    u_as_tensor =self.to_tensor(u_as_img)
    p_as_tensor =self.to_tensor(p_as_img)
    s_as_tensor =self.to_tensor(s_as_img)
    

    _label=torch.tensor(float(self.label_arr[idx]))
    label=torch.reshape(_label,(-1,))
 
    
    return(u_as_tensor,p_as_tensor,s_as_tensor,label )



In [68]:
'''

transfer learning 


새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우
데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다.
새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.

새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우
새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.

새로 훈련할 데이터가 적으며 original 데이터와 다른 경우
데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.

새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우
데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

'''

from torchvision import models

model_pt=models.vgg16(pretrained=True)


for p in model_pt.parameters():
  p.requires_grad=False # 가중치가 학습되지 않도록 함 
#in_features?


print(model_pt.features)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [69]:

'''
pretrained model VGG 16 VGG 19 AlexNet LeNet .... Transfer learning 
'''
import torch.nn.functional as F
from torch import nn
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.l1   =  nn.Linear(98304 ,1024)
        self.l2   =  nn.Linear(1024,1) # regression 이기 때문에 마지막 차원 1 
        self.VGG16 = model_pt.features
        
    def forward(self, top ,bot,shoes):
        top= self.VGG16(top)   #216*216
        bot = self.VGG16(bot)  #
        shoes = self.VGG16(shoes) #
        #outer=self.VGG16(outer)   #
        N ,_,_,_ = top.size()
        top = top.view(N,-1)
        bot = bot.view(N,-1)
        shoes = shoes.view(N,-1)
        #outer = outer.view(N,-1)
        z = torch.cat((top,bot,shoes),1)
        z = self.l1(z)
        z = self.relu(z)
        z = self.l2(z)
        return z
    


In [70]:
model=Model().to(device)

In [71]:
criterion =torch.nn.MSELoss()
optimizer =torch.optim.SGD(model.parameters(), lr=  0.01)

In [72]:
df=pd.read_csv("/content/drive/MyDrive/캡스톤/label_and_path_withoutNaN")
df.iloc[:,6]

0          /content/drive/MyDrive/캡스톤/data/1
1          /content/drive/MyDrive/캡스톤/data/3
2          /content/drive/MyDrive/캡스톤/data/4
3          /content/drive/MyDrive/캡스톤/data/6
4          /content/drive/MyDrive/캡스톤/data/7
                          ...                    
2286    /content/drive/MyDrive/캡스톤/data/7078
2287    /content/drive/MyDrive/캡스톤/data/7080
2288    /content/drive/MyDrive/캡스톤/data/7081
2289    /content/drive/MyDrive/캡스톤/data/7082
2290    /content/drive/MyDrive/캡스톤/data/7092
Name: img_path, Length: 2291, dtype: object

In [77]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split


custom_cloth = MU("/content/drive/MyDrive/캡스톤/label_and_path_withoutNaN")
    # Define dataloader

/content/drive/MyDrive/캡스톤/
dataset_size = len(custom_cloth)
train_size = int(dataset_size * 0.8)

test_size = dataset_size - train_size

train_dataset,  test_dataset = random_split(custom_cloth, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)


In [74]:
"""
def test_prediction(model,  u,p,s ):
    # generating predictions for test set
    with torch.no_grad():
        output = model(u,p,s)

    softmax = torch.exp(output)
    prob = list(softmax)
    prob=torch.Tensor(prob)
    predictions =torch.argmax(prob) # argmax (최대값의 색인을 리턴)

    return predictions
"""
def train(n_epochs,model, optimizer,criterion):
    rmse_history = []
    for epoch in range(1, n_epochs + 1):
        
        dataiter=iter(train_dataloader)
        u,p,s ,label = dataiter.next()
        # clearing the Gradients of the model parameters
        optimizer.zero_grad()

        # prediction for training and validation set
        model.train()
        u=u.type(torch.cuda.FloatTensor)
        p=p.type(torch.cuda.FloatTensor)
        s=s.type(torch.cuda.FloatTensor)
        label=label.to(device)
        output_train = model(u,p,s)

        # computing the training and validation loss
        loss_train = criterion(output_train, label)
        loss_train.backward()
        optimizer.step()

        if epoch % 10== 0:
            model.eval()
            test_loss = 0.

            testiter=iter(test_dataloader)
            u,p,s ,y_test=testiter.next()
            y_test=y_test.detach().cpu().numpy()    
            u=u.type(torch.cuda.FloatTensor)
            p=p.type(torch.cuda.FloatTensor)
            s=s.type(torch.cuda.FloatTensor)   
           # prediction on testing dataset
            pred = model(u,p,s)
            pred=pred.detach().cpu().numpy()
            # evaluate the prediction accuracy
            rmse = math.sqrt(mean_squared_error(y_test,pred))
            print('Epoch :', epoch, '\t', 'loss :', round(loss_train.item(), 3), '\t',
                  "RMSE =", rmse)
            rmse_history.append(rmse)

    return rmse_history

In [78]:
rmse_list=list()
rmse_list=train(100,model,optimizer=optimizer,criterion=criterion)
print(rmse_list)

Epoch : 10 	 loss : 6.857 	 RMSE = 3.716202576633326
Epoch : 20 	 loss : 6.325 	 RMSE = 1.8402587681714186
Epoch : 30 	 loss : 3.157 	 RMSE = 2.1963455104740226
Epoch : 40 	 loss : 3.19 	 RMSE = 1.1257560097103847
Epoch : 50 	 loss : 2.253 	 RMSE = 0.8929702598579038
Epoch : 60 	 loss : 0.954 	 RMSE = 1.6247834281388065
Epoch : 70 	 loss : 1.395 	 RMSE = 0.7388136220861687
Epoch : 80 	 loss : 0.495 	 RMSE = 1.2499738213658862
Epoch : 90 	 loss : 0.662 	 RMSE = 1.1342626826060933
Epoch : 100 	 loss : 0.938 	 RMSE = 1.4311033648540579
[3.716202576633326, 1.8402587681714186, 2.1963455104740226, 1.1257560097103847, 0.8929702598579038, 1.6247834281388065, 0.7388136220861687, 1.2499738213658862, 1.1342626826060933, 1.4311033648540579]


In [79]:
torch.save(model.state_dict(), "/content/drive/MyDrive/캡스톤/FirstModel")

## 시각화

In [80]:
import matplotlib.pyplot as plt
import seaborn as sns